# Preprocessing 01 - Data Preparation - Convert EEG .fif files to .set and .vhdr files

## Import libraries

In [1]:
import os
import mne
from pybv import write_brainvision
import numpy as np
import regex as re
from tqdm.notebook import tqdm

## Set Input directory, Path to Montage, and Channels to drop

In [2]:
filepath_out: str = "./data/eeg_tmp/raw"
montage_path = "./data/EEG/AC-64.bvef"
channels_to_drop = ['x_dir', 'y_dir', 'z_dir']

## Helper Functions For Transorming EEG .fif files to .set and .vhdr files

In [3]:
def load_fif_file(filename, filepath):
    full_filename = os.path.join(filepath, filename)
    return mne.io.read_raw_fif(fname=full_filename + '.fif', preload=True)


def convert_fif_to_set(filename, load_from, save_to, number):
    # load fif file
    raw = load_fif_file(filename, load_from)

    # load and set montage
    montage = mne.channels.read_custom_montage(montage_path, head_size=0.085)
    raw.set_montage(montage)

    # drop channels that wont be used
    raw.drop_channels(channels_to_drop)

    # save data to eeglab format
    savepath: str = os.path.join(save_to, filename + "_" + number + ".set")
    mne.export.export_raw(savepath, raw, 'eeglab', 'auto', overwrite=True, verbose="ERROR")

    # transform events to eeglab format
    events = raw.info["events"]
    np_e = []
    for e in events:
        np_e.append([e['list'][0], e['list'][2]])

    np_array = np.asarray(np_e)

    # save events to eeglab format
    channels = raw.ch_names
    df = raw.get_data(channels)
    filename = filename + "_" + number
    write_brainvision(data=df, sfreq=500, ch_names=channels, folder_out=filepath_out, fname_base=filename, events=np_array, overwrite=True)

## Transform EEG .fif files to .set and .vhdr files

In [4]:
# get all folders in data/filteredData
folders = [f.path for f in os.scandir('./data/rawData') if f.is_dir()]
# extract the numbers
folders = [f.split('/')[-1] for f in folders]
numbers = [int(re.findall(r'\d+', f)[0]) for f in folders]
# numbers to str with leading zeros
numbers = [str(n).zfill(2) for n in numbers]



# iterate over all folders
for number in tqdm(numbers):
    # construct path to folder
    load_from = "./data/rawData/Participant" + number
    save_to = "./data/eeg_tmp/raw"
    file_name = "eeg_raw"
    # transform file via side effect
    convert_fif_to_set(file_name, load_from, save_to, number)


  0%|          | 0/38 [00:00<?, ?it/s]

Opening raw data file ./data/rawData/Participant01\eeg_raw.fif...
Isotrak not found
    Range : 0 ... 2036249 =      0.000 ...  4072.498 secs
Ready.
Reading 0 ... 2036249  =      0.000 ...  4072.498 secs...


C:\Users\jonas\AppData\Local\Temp/ipykernel_20272/3093821844.py:12: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage)


Overwriting existing file.
Opening raw data file ./data/rawData/Participant02\eeg_raw.fif...
Isotrak not found
    Range : 0 ... 2097849 =      0.000 ...  4195.698 secs
Ready.
Reading 0 ... 2097849  =      0.000 ...  4195.698 secs...


C:\Users\jonas\AppData\Local\Temp/ipykernel_20272/3093821844.py:12: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage)


...

C:\Users\jonas\AppData\Local\Temp/ipykernel_20272/3093821844.py:12: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage)


Opening raw data file ./data/rawData/Participant70\eeg_raw.fif...
Isotrak not found
    Range : 0 ... 2222109 =      0.000 ...  4444.218 secs
Ready.
Reading 0 ... 2222109  =      0.000 ...  4444.218 secs...


C:\Users\jonas\AppData\Local\Temp/ipykernel_20272/3093821844.py:12: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage)


Opening raw data file ./data/rawData/Participant71\eeg_raw.fif...
Isotrak not found
    Range : 0 ... 1767489 =      0.000 ...  3534.978 secs
Ready.
Reading 0 ... 1767489  =      0.000 ...  3534.978 secs...


C:\Users\jonas\AppData\Local\Temp/ipykernel_20272/3093821844.py:12: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage)
